# Chapter 19: CI Fundamentals

Continuous Integration (CI) represents the foundational practice of modern DevOps, where developers frequently merge code changes into a central repository, followed by automated builds and tests. While Chapter 11 addressed how container images are stored and distributed, this chapter examines the systematic automation that produces those images. CI transforms manual, error-prone integration processes into reliable, repeatable pipelines that provide rapid feedback on code quality and integration health.

Understanding CI workflow mechanics—from trigger mechanisms through artifact management to failure handling—is essential for implementing pipelines that scale with team size and codebase complexity while maintaining the rapid feedback loops necessary for agile development.

## 19.1 The CI Workflow

The CI workflow orchestrates the journey from source code commit to validated, packaged artifact ready for deployment. This workflow automates the integration verification that historically occurred manually during "integration hell" phases in traditional software development.

### Core CI Principles

**Maintain a Single Source Repository:**
All code, configuration, and dependencies reside in version control (typically Git). This ensures reproducibility—any previous state of the software can be reconstructed precisely.

**Automate the Build:**
The build process must be fully automated, requiring no manual intervention to compile, package, or prepare the application. This includes:
- Dependency resolution
- Compilation or transpilation
- Static asset generation
- Container image building
- Documentation generation

**Make the Build Self-Testing:**
Every build includes automated verification through unit tests, integration tests, and static analysis. The build fails if any verification step fails, preventing defective code from progressing.

**Commit Frequently:**
Developers integrate changes into the main branch at least daily, preferably multiple times per day. Frequent commits reduce integration complexity by minimizing the divergence between individual developer branches and the mainline.

**Fix Broken Builds Immediately:**
A broken CI pipeline receives priority attention over new feature development. The team maintains a "green build" policy where the main branch remains deployable at all times.

### Standard CI Pipeline Flow

```mermaid
graph LR
    A[Code Commit] --> B[Source Checkout]
    B --> C[Dependency Install]
    C --> D[Static Analysis]
    D --> E[Unit Tests]
    E --> F[Build Artifacts]
    F --> G[Integration Tests]
    G --> H[Security Scan]
    H --> I[Artifact Publish]
    I --> J[Notify Team]
```

**Stage Details:**

1. **Source Checkout**: Clone repository and checkout specific commit SHA
2. **Environment Setup**: Configure runtime, install tools, authenticate to services
3. **Dependency Resolution**: Download libraries, verify checksums, cache for performance
4. **Static Analysis**: Linting, code formatting checks, complexity analysis
5. **Compilation/Build**: Transform source code into executable artifacts
6. **Unit Testing**: Fast, isolated tests validating individual components (target: <10 minutes)
7. **Artifact Creation**: Package application (JAR, container image, binary)
8. **Integration Testing**: Verify component interactions with real dependencies
9. **Security Scanning**: Vulnerability assessment of dependencies and container images
10. **Artifact Publication**: Push validated artifacts to repositories (Docker registry, Maven Central, npm)
11. **Notification**: Report status to team (Slack, email, PR comments)

### Pipeline as Code

Modern CI systems define pipelines within the repository itself, versioning the build process alongside the application code:

```yaml
# .github/workflows/ci.yml (GitHub Actions example)
name: Continuous Integration

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

jobs:
  build-and-test:
    runs-on: ubuntu-22.04
    steps:
      - name: Checkout Source
        uses: actions/checkout@v4
        with:
          fetch-depth: 0  # Full history for proper versioning

      - name: Setup Node.js
        uses: actions/setup-node@v4
        with:
          node-version: '20'
          cache: 'npm'

      - name: Install Dependencies
        run: npm ci  # Clean install from package-lock.json

      - name: Lint Code
        run: npm run lint

      - name: Run Unit Tests
        run: npm test -- --coverage --coverageReporters=text-summary

      - name: Build Application
        run: npm run build

      - name: Build Container Image
        run: docker build -t myapp:${{ github.sha }} .

      - name: Scan for Vulnerabilities
        uses: aquasecurity/trivy-action@master
        with:
          image-ref: 'myapp:${{ github.sha }}'
          format: 'sarif'
          output: 'trivy-results.sarif'

      - name: Upload Coverage
        uses: codecov/codecov-action@v3
        with:
          files: ./coverage/lcov.info
```

**Benefits of Pipeline as Code:**
- **Version Control**: Build changes tracked in Git with full audit trail
- **Code Review**: Pipeline modifications undergo peer review via pull requests
- **Branch Isolation**: Different branches can have different build requirements
- **Reproducibility**: Exact build steps documented and executable locally

## 19.2 Trigger Mechanisms

CI pipelines initiate through various trigger mechanisms, each suited to different workflow requirements and integration patterns.

### Git-Based Triggers

**Push Events:**
Trigger on every commit pushed to specific branches or tags:

```yaml
# GitLab CI example
workflow:
  rules:
    - if: $CI_COMMIT_BRANCH == "main"
    - if: $CI_COMMIT_BRANCH == "develop"
    - if: $CI_PIPELINE_SOURCE == "merge_request_event"
```

**Pull Request/Merge Request Triggers:**
Execute pipelines when code is proposed for integration, validating changes before they reach mainline:

```yaml
# Jenkinsfile (Declarative Pipeline)
pipeline {
    triggers {
        githubPullRequest {
            spec('* * * * *')
            cron('H/5 * * * *')
            permitAll()
            extensions {
                commitStatus {
                    context('continuous-integration/jenkins/pr')
                    addTestResults(true)
                }
            }
        }
    }
}
```

**Tagging Releases:**
Trigger specialized release pipelines when version tags are pushed:

```yaml
# Azure DevOps
trigger:
  tags:
    include:
      - v*
  branches:
    include:
      - main
```

### Scheduled Triggers (Cron)

Run pipelines at scheduled intervals for:
- Nightly integration tests
- Dependency vulnerability scans
- Cleanup tasks
- Performance regression testing

```yaml
# GitHub Actions Scheduled Workflow
on:
  schedule:
    # Run at 2 AM UTC daily
    - cron: '0 2 * * *'
    
  workflow_dispatch:  # Allow manual triggering
    inputs:
      environment:
        description: 'Environment to deploy'
        required: true
        default: 'staging'
        type: choice
        options:
          - staging
          - production
```

**Cron Syntax Reference:**
```
* * * * *
│ │ │ │ └─── Day of week (0-7, where 0 and 7 = Sunday)
│ │ │ └───── Month (1-12)
│ │ └─────── Day of month (1-31)
│ └───────── Hour (0-23)
└─────────── Minute (0-59)

Examples:
H/15 * * * *    Every 15 minutes (H = hash for load distribution)
0 2 * * *       Daily at 2 AM
0 0 * * 0       Weekly on Sunday
0 0 1 * *       Monthly on the 1st
```

### Webhook Triggers

External systems trigger pipelines via HTTP POST requests:

```bash
# Trigger Jenkins pipeline remotely
curl -X POST \
  -H "Authorization: Bearer $TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"parameter": [{"name":"ENVIRONMENT", "value":"staging"}]}' \
  https://jenkins.company.com/job/deploy/buildWithParameters
```

**Common Webhook Sources:**
- **Slack**: `/deploy production` command triggers pipeline
- **Jira**: Issue transition triggers deployment to testing environment
- **Monitoring**: Alertmanager webhook triggers rollback pipeline
- **Artifact Repository**: New base image availability triggers rebuild

### Path-Based Triggers

Optimize CI resources by running only relevant pipelines when specific files change:

```yaml
# GitHub Actions with path filters
on:
  push:
    paths:
      - 'src/**'
      - 'package*.json'
      - 'Dockerfile'
      - '.github/workflows/ci.yml'
    paths-ignore:
      - '**/*.md'
      - 'docs/**'
```

**Monorepo Strategy:**
In monorepos, trigger different pipelines based on changed directories:

```yaml
# Azure DevOps with path filters
trigger:
  branches:
    include:
      - main
  paths:
    include:
      - services/payment/**

stages:
- stage: BuildPaymentService
  jobs:
  - job: Build
    steps:
    - script: |
        if git diff --name-only HEAD~1 | grep -q "^services/payment/"; then
          echo "Payment service changed, building..."
          cd services/payment && docker build .
        fi
```

## 19.3 Pipeline Stages

Pipeline stages group related steps into logical phases, each with specific responsibilities and success criteria. Well-defined stages improve readability, enable conditional execution, and facilitate parallelization.

### Stage Structure

```yaml
# GitLab CI Pipeline Stages
stages:
  - validate      # Linting, formatting, security scanning
  - build         # Compilation, packaging, image creation
  - test          # Unit, integration, e2e tests
  - security      # SAST, DAST, dependency scanning
  - publish       # Artifact publication, registry push
  - deploy        # Environment deployment (often in CD phase)

variables:
  DOCKER_DRIVER: overlay2
  DOCKER_TLS_CERTDIR: "/certs"

validate:lint:
  stage: validate
  image: node:20-alpine
  script:
    - npm ci
    - npm run lint:strict
    - npm run type-check
  rules:
    - if: $CI_PIPELINE_SOURCE == "merge_request_event"

build:docker:
  stage: build
  image: docker:24
  services:
    - docker:24-dind
  script:
    - docker build --pull -t $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA .
    - docker push $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
  dependencies: []

test:unit:
  stage: test
  image: node:20-alpine
  coverage: '/All files[^|]*\|[^|]*\s+([\d\.]+)/'
  script:
    - npm ci
    - npm run test:ci
  artifacts:
    reports:
      coverage_report:
        coverage_format: cobertura
        path: coverage/cobertura-coverage.xml
      junit: junit.xml
    paths:
      - coverage/

security:dependency_check:
  stage: security
  image: owasp/dependency-check:latest
  script:
    - /usr/share/dependency-check/bin/dependency-check.sh 
      --project "$CI_PROJECT_NAME" 
      --scan . 
      --format JSON 
      --failOnCVSS 7
  allow_failure: false

publish:helm_chart:
  stage: publish
  image: alpine/helm:latest
  script:
    - helm package ./chart
    - curl --data-binary "@myapp-1.0.0.tgz" $HELM_REPO_URL
  only:
    - tags
```

### Stage Gates and Approvals

Prevent progression between stages without manual or automated verification:

```yaml
# Azure DevOps with environment approvals
stages:
- stage: Build
  jobs:
  - job: Compile
    steps:
    - script: echo "Building..."

- stage: DeployToStaging
  dependsOn: Build
  jobs:
  - deployment: StagingDeploy
    environment: 'staging'
    strategy:
      runOnce:
        deploy:
          steps:
          - script: echo "Deploying to staging"

- stage: DeployToProduction
  dependsOn: DeployToStaging
  condition: succeeded()
  jobs:
  - deployment: ProductionDeploy
    environment: 'production'  # Requires approval configured in Azure DevOps
    strategy:
      canary:
        increments: [25, 50, 100]
```

### Conditional Stage Execution

Execute stages based on context:

```yaml
# Jenkinsfile with conditional stages
pipeline {
    agent any
    
    stages {
        stage('Build') {
            steps {
                sh 'make build'
            }
        }
        
        stage('Integration Tests') {
            when {
                anyOf {
                    branch 'main'
                    branch 'develop'
                    changeRequest()  // Pull requests
                }
            }
            steps {
                sh 'make integration-test'
            }
        }
        
        stage('Performance Tests') {
            when {
                allOf {
                    branch 'main'
                    triggeredBy 'TimerTrigger'  // Only on scheduled runs
                }
            }
            steps {
                sh 'make perf-test'
            }
        }
        
        stage('Deploy') {
            when {
                expression { 
                    return env.GIT_TAG_NAME != null 
                }
            }
            steps {
                sh 'make deploy'
            }
        }
    }
}
```

## 19.4 Artifact Management

Artifacts represent the tangible outputs of CI pipelines—compiled binaries, container images, documentation, test reports, and deployment packages. Proper artifact management ensures traceability, reproducibility, and efficient distribution.

### Artifact Types

**Build Artifacts:**
- Compiled binaries (JAR, WAR, EXE)
- Container images
- Generated code and documentation
- Configuration packages (Helm charts, Terraform modules)

**Test Artifacts:**
- Coverage reports (HTML, XML)
- Test result files (JUnit XML, TRX)
- Screenshots and videos from UI tests
- Performance test reports

**Metadata:**
- SBOMs (Software Bill of Materials)
- Build provenance attestations
- Vulnerability scan reports

### Artifact Repository Integration

**Binary Repositories (Nexus, Artifactory):**
```yaml
# Upload artifacts to Nexus (Maven example)
- script: |
    mvn deploy \
      -DaltDeploymentRepository=nexus::default::https://nexus.company.com/repository/maven-releases/ \
      -DskipTests
  env:
    MAVEN_USERNAME: $(NEXUS_USER)
    MAVEN_PASSWORD: $(NEXUS_PASS)
```

**Container Registries:**
As detailed in Chapter 11, push images with immutable tags:
```bash
docker tag myapp:$BUILD_NUMBER myapp:$GIT_COMMIT
docker push myapp:$GIT_COMMIT
docker push myapp:latest  # Only if this is the latest stable
```

**Generic Artifact Storage (S3, GCS, Azure Blob):**
```yaml
# Upload build artifacts to S3 with metadata
- aws s3 cp ./dist/ s3://my-ci-artifacts/$BUILD_NUMBER/ \
    --recursive \
    --metadata commit-sha=$GIT_COMMIT,branch=$BRANCH_NAME
    
# Tag as latest for easy retrieval
- aws s3 sync ./dist/ s3://my-ci-artifacts/latest/ \
    --delete
```

### Artifact Retention Policies

Prevent storage cost explosion and security accumulation of stale artifacts:

```yaml
# GitLab CI artifact retention
artifacts:
  name: "$CI_JOB_STAGE-$CI_COMMIT_REF_NAME"
  paths:
    - dist/
  reports:
    junit: test-results.xml
  expire_in: 1 week  # Auto-delete after 7 days
  when: always       # Archive even on failure (for debugging)

# For long-term storage, push to dedicated artifact repository
publish-to-s3:
  script:
    - aws s3 cp dist/app.tar.gz s3://permanent-artifacts/$CI_COMMIT_SHA/app.tar.gz
    - aws s3 cp dist/app.tar.gz s3://permanent-artifacts/latest/app.tar.gz
  only:
    - main
    - tags
```

**Retention Best Practices:**
- **Short-term** (1-7 days): Build logs, temporary test outputs, debug symbols
- **Medium-term** (30-90 days): Release candidates, installer packages
- **Long-term** (years): Released versions, compliance documentation, SBOMs

### Artifact Immutability

Once published, artifacts must never be modified—only superseded by new versions:

```bash
# Enable immutability on S3 bucket
aws s3api put-object-lock-configuration \
  --bucket my-ci-artifacts \
  --object-lock-configuration ObjectLockEnabled=Enabled,Rule={DefaultRetention={Mode=COMPLIANCE,Days=1}}

# Prevent overwrites in Azure Blob
az storage blob upload \
  --file app.tar.gz \
  --name builds/$BUILD_NUMBER/app.tar.gz \
  --container-name artifacts \
  --if-none-match "*"  # Fail if exists
```

## 19.5 Parallel Execution

Parallel execution reduces pipeline duration by distributing independent work across multiple compute resources, critical for maintaining rapid feedback loops as codebases grow.

### Parallel Jobs

Execute independent jobs simultaneously:

```yaml
# GitHub Actions Matrix Strategy
jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        test-suite: [unit, integration, e2e]
        node-version: [18, 20]
        include:
          - test-suite: e2e
            browser: chrome
        exclude:
          - test-suite: unit
            node-version: 18  # Skip redundant combinations
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Node ${{ matrix.node-version }}
        uses: actions/setup-node@v4
        with:
          node-version: ${{ matrix.node-version }}
      
      - name: Run ${{ matrix.test-suite }} tests
        run: npm run test:${{ matrix.test-suite }}
```

**Resource Considerations:**
- Parallelism consumes more CI runner resources simultaneously
- Consider concurrency limits to prevent overwhelming shared services (databases, APIs)
- Use service containers with unique ports for isolation

### Fan-Out/Fan-In Patterns

Distribute work to parallel workers, then aggregate results:

```yaml
# Azure DevOps Fan-Out/Fan-In
stages:
- stage: Build
  jobs:
  - job: Compile
    steps:
    - script: echo "Compiling..."
    - publish: $(Build.ArtifactStagingDirectory)
      artifact: compiled-code

- stage: Test
  dependsOn: Build
  jobs:
  - job: UnitTests
    steps:
    - download: current
      artifact: compiled-code
    - script: echo "Running unit tests..."
    
  - job: IntegrationTests
    steps:
    - download: current
      artifact: compiled-code
    - script: echo "Running integration tests..."
    
  - job: SecurityScan
    steps:
    - download: current
      artifact: compiled-code
    - script: echo "Scanning..."

- stage: Deploy
  dependsOn: Test  # Waits for all Test jobs
  jobs:
  - deployment: Production
    steps:
    - script: echo "Deploying..."
```

### Test Splitting

Divide large test suites across multiple runners:

```bash
# CircleCI Test Splitting
test:
  parallelism: 4  # Split across 4 containers
  steps:
    - run:
        command: |
          # Get test files and split by timing data (runs slow tests first)
          TEST_FILES=$(circleci tests glob "tests/**/*.test.js" | circleci tests split --split-by=timings)
          
          # Run only this container's portion
          npm test -- $TEST_FILES
          
    - store_test_results:
        path: test-results
```

**Splitting Strategies:**
- **File size**: Distribute files evenly (simple, unbalanced if test durations vary)
- **Timing data**: Historical duration data balances load optimally
- **Round-robin**: Simple distribution without historical data

### Resource Constraints

Prevent parallel execution from overwhelming infrastructure:

```yaml
# GitLab CI concurrency control
test:integration:
  parallel: 5
  resource_group: integration_database  # Only 1 job accesses DB at a time
  script:
    - npm run test:integration

deploy:staging:
  resource_group: staging_environment    # Prevent concurrent deployments
  script:
    - helm upgrade --install myapp ./chart
```

## 19.6 Caching Strategies

Caching reuses data between pipeline runs to accelerate builds. Effective caching significantly reduces pipeline duration and dependency on external networks.

### Cache vs. Artifacts

**Cache:**
- Reused between pipeline runs
- Stored on runner/host (ephemeral)
- Example: Downloaded npm packages, compiled dependencies
- Can be deleted without losing critical data (regenerable)

**Artifacts:**
- Passed between stages of the same pipeline
- Stored in artifact repository (persistent)
- Example: Compiled binaries, test reports
- Must be retained for compliance/debugging

### Dependency Caching

Cache package manager directories between builds:

```yaml
# GitHub Actions Caching
- uses: actions/cache@v3
  with:
    path: |
      ~/.npm
      ~/.m2
      ~/.gradle/caches
      ~/.cache/pip
    key: ${{ runner.os }}-deps-${{ hashFiles('**/package-lock.json', '**/pom.xml', '**/requirements.txt') }}
    restore-keys: |
      ${{ runner.os }}-deps-
      ${{ runner.os }}-
```

**Cache Key Strategy:**
- **Primary key**: Hash of lock files (package-lock.json, yarn.lock, Cargo.lock)
- **Restore keys**: Fallback to less specific keys if exact match unavailable
- **OS specificity**: Include runner OS in key (Linux/macOS/Windows paths differ)

### Docker Layer Caching

Reuse previously built image layers:

```yaml
# GitHub Actions with BuildKit cache
- name: Build and push
  uses: docker/build-push-action@v5
  with:
    context: .
    push: true
    tags: myapp:latest
    cache-from: type=gha  # GitHub Actions cache backend
    cache-to: type=gha,mode=max
```

**Advanced Docker Caching (Registry-based):**
```bash
# Pull previous image to use as cache source
docker pull myapp:latest || true

# Build with cache from registry
docker build \
  --cache-from myapp:latest \
  --tag myapp:$BUILD_NUMBER \
  --tag myapp:latest \
  .

# Push new layers
docker push myapp:$BUILD_NUMBER
docker push myapp:latest
```

### Cache Invalidation

Proper cache invalidation prevents stale dependencies:

```yaml
# GitLab CI cache configuration
cache:
  key:
    files:
      - package-lock.json
      - Gemfile.lock
  paths:
    - node_modules/
    - vendor/
  policy: pull-push  # Download at start, upload at end

# Force cache refresh monthly (prevent indefinite stale caches)
variables:
  CACHE_VERSION: "2024-01"
  
cache:
  key: "$CACHE_VERSION-$CI_COMMIT_REF_SLUG"
```

**Invalidation Triggers:**
- Lock file changes (automatic via hash keys)
- Scheduled cache refresh (weekly/monthly)
- Manual cache purge when corruption suspected

### Distributed Caching

For large teams, use centralized cache storage:

```yaml
# Jenkins with S3-backed cache
pipeline {
    agent any
    
    environment {
        AWS_DEFAULT_REGION = 'us-east-1'
    }
    
    stages {
        stage('Build') {
            steps {
                cache(caches: [
                    arbitraryFileCache(
                        path: 'node_modules',
                        cacheValidityDecidingFile: 'package-lock.json',
                        cacheDirectory: 's3://my-jenkins-cache/$JOB_NAME/'
                    )
                ]) {
                    sh 'npm ci'
                    sh 'npm run build'
                }
            }
        }
    }
}
```

## 19.7 Failure Handling

Robust CI pipelines handle failures gracefully, providing actionable diagnostics and preventing resource waste.

### Fast-Fail Strategies

Terminate pipelines quickly when critical steps fail:

```yaml
# GitLab CI fast-fail
stages:
  - validate
  - build
  - test

job:lint:
  stage: validate
  script:
    - npm run lint
  allow_failure: false  # Fail immediately, block subsequent stages

job:unit_tests:
  stage: test
  parallel:
    matrix:
      - TEST_GROUP: [1, 2, 3, 4]
  script:
    - npm test -- --group=$TEST_GROUP
  timeout: 10 minutes    # Kill if hung
  retry: 2               # Retry flaky tests twice
  after_script:          # Always run cleanup
    - npm run test:cleanup
```

**Failure Behavior:**
- **Hard fail**: Stop entire pipeline immediately (default for compile errors)
- **Soft fail**: Continue pipeline but mark as unstable (acceptable for non-critical warnings)
- **Retry**: Automatic retry for transient failures (network timeouts, flaky tests)

### Notification Strategies

Alert relevant teams promptly when failures occur:

```yaml
# GitHub Actions notifications
- name: Notify Slack on Failure
  if: failure()
  uses: 8398a7/action-slack@v3
  with:
    status: ${{ job.status }}
    channel: '#ci-alerts'
    webhook_url: ${{ secrets.SLACK_WEBHOOK }}
    fields: repo,message,commit,author,action,eventName,ref,workflow
    
- name: Create GitHub Issue for Main Branch Failure
  if: failure() && github.ref == 'refs/heads/main'
  uses: actions/github-script@v7
  with:
    script: |
      github.rest.issues.create({
        owner: context.repo.owner,
        repo: context.repo.repo,
        title: `CI Failure: ${context.workflow} - ${context.sha}`,
        body: `Pipeline failed on main branch. [View logs](${context.serverUrl}/${context.repo.owner}/${context.repo.repo}/actions/runs/${context.runId})`,
        labels: ['ci-failure', 'priority-high']
      })
```

### Debugging Failed Builds

Preserve state for investigation:

```yaml
# Upload artifacts even on failure
artifacts:
  when: always  # Upload logs even if job fails
  paths:
    - logs/
    - core-dumps/
    - test-screenshots/
  expire_in: 1 week

# Enable SSH access for debugging (GitHub Actions)
- name: Setup tmate session
  if: failure() && github.event.inputs.debug == 'true'
  uses: mxschmitt/action-tmate@v3
  timeout-minutes: 30
  with:
    limit-access-to-actor: true
```

**Diagnostic Artifacts:**
- Full build logs (verbose mode)
- Environment variable dumps (sanitized)
- Dependency tree listings
- System resource utilization (CPU/memory)

### Cleanup and Resource Management

Ensure resources are released even on failure:

```bash
#!/bin/bash
# cleanup.sh - Runs on EXIT trap

cleanup() {
  echo "Cleaning up resources..."
  docker-compose down -v || true
  kubectl delete namespace test-$BUILD_NUMBER --ignore-not-found=true || true
  rm -f temp-credentials.json || true
}

trap cleanup EXIT ERR

# Main build steps...
```

## 19.8 CI Metrics

Measuring CI performance enables continuous improvement of pipeline efficiency and reliability.

### DORA Metrics Integration

The DevOps Research and Assessment (DORA) team identified four key metrics, two directly measurable in CI:

**Deployment Frequency:**
Count successful pipeline executions deploying to production:

```sql
-- Example query for monitoring database
SELECT 
  DATE(created_at) as date,
  COUNT(*) as deployment_count
FROM pipeline_runs
WHERE status = 'success' 
  AND target_environment = 'production'
GROUP BY DATE(created_at)
ORDER BY date;
```

**Change Lead Time:**
Measure duration from code commit to production deployment:

```yaml
# Track timestamps in pipeline
steps:
  - id: start-time
    run: echo "start=$(date +%s)" >> $GITHUB_OUTPUT
  
  # ... build and test steps ...
  
  - id: end-time
    run: |
      end=$(date +%s)
      start=${{ steps.start-time.outputs.start }}
      lead_time=$((end - start))
      echo "Lead time: ${lead_time}s"
      
      # Send to monitoring
      curl -X POST $METRICS_API \
        -d "{\"lead_time_seconds\": $lead_time, \"commit\": \"$GITHUB_SHA\"}"
```

### Pipeline Performance Metrics

**Duration Trends:**
Track stage timing to identify bottlenecks:

```python
# Python script to analyze Jenkins build times
import jenkins
import pandas as pd

server = jenkins.Jenkins('http://jenkins:8080', username='user', password='pass')
job_name = 'my-pipeline'

builds = server.get_job_info(job_name)['builds']
data = []

for build in builds[:50]:  # Last 50 builds
    build_info = server.get_build_info(job_name, build['number'])
    for stage in build_info['stages']:
        data.append({
            'build': build['number'],
            'stage': stage['name'],
            'duration': stage['durationMillis'] / 1000,
            'status': stage['status']
        })

df = pd.DataFrame(data)
pivot = df.pivot(index='build', columns='stage', values='duration')
pivot.plot(kind='line', figsize=(12, 6))  # Visualize trends
```

**Key Metrics to Track:**
- **Queue time**: Duration waiting for available runner
- **Setup time**: Environment preparation (cloning, tool installation)
- **Build time**: Compilation and packaging
- **Test time**: Test execution duration
- **Success rate**: Percentage of successful builds (target: >90%)
- **Flakiness**: Tests with inconsistent results across identical runs

### Metric Visualization

Export metrics to monitoring systems:

```yaml
# GitLab CI metrics export
metrics:
  pipeline: |
    # HELP ci_pipeline_duration_seconds Total pipeline duration
    # TYPE ci_pipeline_duration_seconds gauge
    ci_pipeline_duration_seconds{pipeline_id="$CI_PIPELINE_ID",status="$CI_PIPELINE_STATUS"} $CI_PIPELINE_DURATION
    
    # HELP ci_job_duration_seconds Individual job duration  
    # TYPE ci_job_duration_seconds gauge
    ci_job_duration_seconds{job_name="$CI_JOB_NAME",stage="$CI_JOB_STAGE"} $CI_JOB_DURATION

# Push to Prometheus Pushgateway
after_script:
  - |
    cat <<EOF | curl --data-binary @- http://pushgateway:9091/metrics/job/ci_pipeline/instance/$CI_PROJECT_PATH
    $metrics
    EOF
```

---

## Chapter Summary and Preview

In this chapter, we established the foundational mechanics of Continuous Integration workflows. We examined trigger mechanisms ranging from Git-based events and scheduled cron jobs to external webhook initiations, enabling precise control over when pipelines execute. Pipeline stages organize work into logical phases—from validation and building to testing and publishing—with gates controlling progression between environments. Artifact management strategies ensure reproducible storage of build outputs while implementing retention policies to control costs. Parallel execution techniques reduce feedback time by distributing independent workloads across multiple runners, while intelligent caching of dependencies and Docker layers prevents redundant work between pipeline runs. Failure handling patterns including fast-fail strategies, comprehensive notifications, and diagnostic artifact preservation ensure teams receive rapid, actionable feedback when integrations fail. Finally, we explored CI metrics aligned with DORA research, measuring deployment frequency and change lead time to drive continuous improvement of pipeline efficiency.

**Key Takeaways:**
- Design pipelines as code committed to version control, ensuring build process changes undergo peer review and maintain audit trails
- Implement immutable artifact storage with strict retention policies to balance accessibility with compliance and cost constraints
- Utilize matrix builds and test splitting to parallelize independent workloads, but implement resource groups to prevent conflicts in shared environments
- Employ hierarchical cache keys (specific hash → branch name → generic fallback) to maximize cache hits while ensuring freshness
- Measure pipeline duration trends and success rates; treat flaky tests with the same urgency as production bugs

**Next Chapter Preview:**
Chapter 20: Git Workflows for CI/CD explores how branching strategies and repository organization patterns integrate with continuous integration practices. We will analyze Feature Branch workflows, GitFlow, Trunk-Based Development, and Pull Request processes, examining how each pattern impacts CI pipeline design, merge conflict resolution, and deployment cadence. Understanding these workflows is essential for structuring CI triggers, defining pipeline scope, and establishing the code review practices that maintain mainline stability while enabling rapid feature development. We will evaluate the trade-offs between long-lived branches and continuous integration frequency, providing guidance on selecting appropriate workflows for team size, release requirements, and regulatory constraints.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../3. kubernetes_fundamentals/18. kubernetes_security.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='20. git_workflows_for_cicd.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
